In [4]:
import zipfile
import os

# Path to the ZIP file
zip_path = "/content/leapGestRecog.zip"  # Replace with the actual path
extract_dir = "/content"  # Directory to extract files

# Extract files from the ZIP archive
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
        print(f"Files successfully extracted to {extract_dir}")
except Exception as e:
    print(f"Error during extraction: {e}")


Files successfully extracted to /content


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Directory containing extracted data
data_dir = extract_dir

# Create a data generator with image augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,          # Normalize pixel values to 0-1
    validation_split=0.2        # Split 20% of data for validation
)

# Load training data
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),       # Resize images to 64x64
    batch_size=32,
    class_mode='categorical',   # Use categorical labels (one-hot encoding)
    subset='training'           # Training subset
)

# Load validation data
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'         # Validation subset
)


Found 1600 images belonging to 3 classes.
Found 400 images belonging to 3 classes.


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Create a Convolutional Neural Network (CNN)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),  # First convolutional layer
    MaxPooling2D(pool_size=(2, 2)),                                 # Max pooling

    Conv2D(64, (3, 3), activation='relu'),                          # Second convolutional layer
    MaxPooling2D(pool_size=(2, 2)),                                 # Max pooling

    Conv2D(128, (3, 3), activation='relu'),                         # Third convolutional layer
    MaxPooling2D(pool_size=(2, 2)),                                 # Max pooling

    Flatten(),                                                     # Flatten the output for the dense layer
    Dense(128, activation='relu'),                                 # Fully connected layer
    Dropout(0.5),                                                  # Dropout to prevent overfitting
    Dense(train_data.num_classes, activation='softmax')            # Output layer with softmax for classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Suitable for multi-class classification
    metrics=['accuracy']
)

model.summary()  # Print the model's architecture


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 683,587 (2.61 MB)

 Trainable params: 683,587 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Train the model on training data and validate on validation data
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,         # Number of epochs (can be adjusted)
    verbose=1          # Display progress during training
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 298ms/step - accuracy: 0.9418 - loss: 0.1578 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 15s 288ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 15s 290ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 298ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 287ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 15s 287ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 284ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 288ms/step - a

In [8]:
# Evaluate the model's performance on the validation data
loss, accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

# Save the trained model for future use
model.save("hand_gesture_recognition_model.h5")
print("Model saved successfully.")


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 186ms/step - accuracy: 1.0000 - loss: 0.0000e+00


Validation Accuracy: 100.00%
Model saved successfully.


In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load a single image for prediction
img_path = "/content/leapGestRecog/03_fist/frame_00_03_0127.png"  # Replace with the path to your image
img = image.load_img(img_path, target_size=(64, 64))  # Resize to match the input shape
img_array = image.img_to_array(img) / 255.0          # Normalize pixel values
img_array = np.expand_dims(img_array, axis=0)        # Add a batch dimension

# Make predictions
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)  # Get the index of the highest probability
print(f"Predicted Gesture: {list(train_data.class_indices.keys())[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted Gesture: leapGestRecog
